In [1]:
import tensorflow as tf
import numpy as np

In [2]:
class MNISTLoader():
    def __init__(self):
        mnist = tf.keras.datasets.mnist
        (self.train_data, self.train_label), (self.test_data, self.test_label) = mnist.load_data()
        # MNIST中的图像默认为uint8（0-255的数字）。以下代码将其归一化到0-1之间的浮点数，并在最后增加一维作为颜色通道
        self.train_data = np.expand_dims(self.train_data.astype(np.float32) / 255.0, axis=-1)      # [60000, 28, 28, 1]
        self.test_data = np.expand_dims(self.test_data.astype(np.float32) / 255.0, axis=-1)        # [10000, 28, 28, 1]
        self.train_label = self.train_label.astype(np.int32)    # [60000]
        self.test_label = self.test_label.astype(np.int32)      # [10000]
        self.num_train_data, self.num_test_data = self.train_data.shape[0], self.test_data.shape[0]

    def get_batch(self, batch_size):
        # 从数据集中随机取出batch_size个元素并返回
        index = np.random.randint(0, np.shape(self.train_data)[0], batch_size)
        return self.train_data[index, :], self.train_label[index]

In [3]:
class CNN(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.conv1 = tf.keras.layers.Conv2D(
            filters=32,             # 卷积层神经元（卷积核）数目
            kernel_size=[5, 5],     # 感受野大小
            padding='same',         # padding策略（vaild 或 same）
            activation=tf.nn.relu   # 激活函数
        )
        self.pool1 = tf.keras.layers.MaxPool2D(pool_size=[2, 2], strides=2)
        self.conv2 = tf.keras.layers.Conv2D(
            filters=64,
            kernel_size=[5, 5],
            padding='same',
            activation=tf.nn.relu
        )
        self.pool2 = tf.keras.layers.MaxPool2D(pool_size=[2, 2], strides=2)
        self.flatten = tf.keras.layers.Reshape(target_shape=(7 * 7 * 64,))
        self.dense1 = tf.keras.layers.Dense(units=1024, activation=tf.nn.relu)
        self.dense2 = tf.keras.layers.Dense(units=10)

    def call(self, inputs):
        x = self.conv1(inputs)                  # [batch_size, 28, 28, 32]
        x = self.pool1(x)                       # [batch_size, 14, 14, 32]
        x = self.conv2(x)                       # [batch_size, 14, 14, 64]
        x = self.pool2(x)                       # [batch_size, 7, 7, 64]
        x = self.flatten(x)                     # [batch_size, 7 * 7 * 64]
        x = self.dense1(x)                      # [batch_size, 1024]
        x = self.dense2(x)                      # [batch_size, 10]
        output = tf.nn.softmax(x)
        return output

In [4]:
num_epochs = 5
batch_size = 50
learning_rate = 0.001

model = CNN()
data_loader = MNISTLoader()
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

In [5]:
num_batches = int(data_loader.num_train_data // batch_size * num_epochs)
for batch_index in range(num_batches):
    X, y = data_loader.get_batch(batch_size)
    with tf.GradientTape() as tape:
        y_pred = model(X)
        loss = tf.keras.losses.sparse_categorical_crossentropy(y_true=y, y_pred=y_pred)
        loss = tf.reduce_mean(loss)
        print("batch %d: loss %f" % (batch_index, loss.numpy()))
    grads = tape.gradient(loss, model.variables)
    optimizer.apply_gradients(grads_and_vars=zip(grads, model.variables))

batch 0: loss 2.297403
batch 1: loss 2.348198
batch 2: loss 2.122819
batch 3: loss 2.051644
batch 4: loss 1.827743
batch 5: loss 1.743684
batch 6: loss 1.581922
batch 7: loss 1.421685
batch 8: loss 1.039261
batch 9: loss 0.884693
batch 10: loss 0.775878
batch 11: loss 0.673978
batch 12: loss 0.871490
batch 13: loss 0.754903
batch 14: loss 1.069770
batch 15: loss 0.702821
batch 16: loss 0.360918
batch 17: loss 1.117430
batch 18: loss 0.542748
batch 19: loss 0.602121
batch 20: loss 0.496368
batch 21: loss 0.702015
batch 22: loss 0.370273
batch 23: loss 0.591100
batch 24: loss 0.403961
batch 25: loss 0.315173
batch 26: loss 0.374566
batch 27: loss 0.619674
batch 28: loss 0.222770
batch 29: loss 0.478206
batch 30: loss 0.579490
batch 31: loss 0.229324
batch 32: loss 0.389028
batch 33: loss 0.266873
batch 34: loss 0.382372
batch 35: loss 0.138884
batch 36: loss 0.315404
batch 37: loss 0.340906
batch 38: loss 0.178223
batch 39: loss 0.178778
batch 40: loss 0.403512
batch 41: loss 0.344115
ba

batch 334: loss 0.040632
batch 335: loss 0.004826
batch 336: loss 0.032228
batch 337: loss 0.018127
batch 338: loss 0.052867
batch 339: loss 0.142938
batch 340: loss 0.053550
batch 341: loss 0.015874
batch 342: loss 0.284057
batch 343: loss 0.112168
batch 344: loss 0.019126
batch 345: loss 0.114174
batch 346: loss 0.038753
batch 347: loss 0.024719
batch 348: loss 0.043909
batch 349: loss 0.034341
batch 350: loss 0.111136
batch 351: loss 0.037535
batch 352: loss 0.264760
batch 353: loss 0.066526
batch 354: loss 0.072548
batch 355: loss 0.056541
batch 356: loss 0.003068
batch 357: loss 0.087458
batch 358: loss 0.168395
batch 359: loss 0.027578
batch 360: loss 0.106611
batch 361: loss 0.007606
batch 362: loss 0.043195
batch 363: loss 0.092137
batch 364: loss 0.033394
batch 365: loss 0.133031
batch 366: loss 0.013506
batch 367: loss 0.066437
batch 368: loss 0.016954
batch 369: loss 0.022692
batch 370: loss 0.031170
batch 371: loss 0.094177
batch 372: loss 0.113477
batch 373: loss 0.023622


batch 662: loss 0.030436
batch 663: loss 0.044334
batch 664: loss 0.020627
batch 665: loss 0.005264
batch 666: loss 0.102340
batch 667: loss 0.136960
batch 668: loss 0.004505
batch 669: loss 0.047934
batch 670: loss 0.050674
batch 671: loss 0.016247
batch 672: loss 0.036117
batch 673: loss 0.070996
batch 674: loss 0.044471
batch 675: loss 0.123752
batch 676: loss 0.009076
batch 677: loss 0.080286
batch 678: loss 0.005128
batch 679: loss 0.075374
batch 680: loss 0.024086
batch 681: loss 0.095356
batch 682: loss 0.050648
batch 683: loss 0.010371
batch 684: loss 0.051231
batch 685: loss 0.093255
batch 686: loss 0.080280
batch 687: loss 0.007537
batch 688: loss 0.072695
batch 689: loss 0.018940
batch 690: loss 0.103373
batch 691: loss 0.009608
batch 692: loss 0.167439
batch 693: loss 0.069065
batch 694: loss 0.092153
batch 695: loss 0.007372
batch 696: loss 0.108465
batch 697: loss 0.016705
batch 698: loss 0.015881
batch 699: loss 0.050164
batch 700: loss 0.013155
batch 701: loss 0.055485


batch 990: loss 0.017226
batch 991: loss 0.065791
batch 992: loss 0.025485
batch 993: loss 0.030300
batch 994: loss 0.029626
batch 995: loss 0.000821
batch 996: loss 0.088556
batch 997: loss 0.032728
batch 998: loss 0.012708
batch 999: loss 0.080330
batch 1000: loss 0.114822
batch 1001: loss 0.065191
batch 1002: loss 0.007103
batch 1003: loss 0.024889
batch 1004: loss 0.066301
batch 1005: loss 0.003314
batch 1006: loss 0.044163
batch 1007: loss 0.022345
batch 1008: loss 0.077293
batch 1009: loss 0.079624
batch 1010: loss 0.141082
batch 1011: loss 0.119501
batch 1012: loss 0.009793
batch 1013: loss 0.102662
batch 1014: loss 0.002167
batch 1015: loss 0.050739
batch 1016: loss 0.008737
batch 1017: loss 0.014453
batch 1018: loss 0.149180
batch 1019: loss 0.032222
batch 1020: loss 0.078939
batch 1021: loss 0.017446
batch 1022: loss 0.035702
batch 1023: loss 0.050921
batch 1024: loss 0.014707
batch 1025: loss 0.006006
batch 1026: loss 0.023241
batch 1027: loss 0.034349
batch 1028: loss 0.019

batch 1306: loss 0.018347
batch 1307: loss 0.048531
batch 1308: loss 0.001137
batch 1309: loss 0.020068
batch 1310: loss 0.036150
batch 1311: loss 0.002695
batch 1312: loss 0.004780
batch 1313: loss 0.003957
batch 1314: loss 0.004036
batch 1315: loss 0.004835
batch 1316: loss 0.006293
batch 1317: loss 0.016671
batch 1318: loss 0.013633
batch 1319: loss 0.323313
batch 1320: loss 0.004282
batch 1321: loss 0.033564
batch 1322: loss 0.012940
batch 1323: loss 0.064398
batch 1324: loss 0.078786
batch 1325: loss 0.000944
batch 1326: loss 0.023400
batch 1327: loss 0.002524
batch 1328: loss 0.023961
batch 1329: loss 0.050463
batch 1330: loss 0.002305
batch 1331: loss 0.043298
batch 1332: loss 0.137117
batch 1333: loss 0.030753
batch 1334: loss 0.002944
batch 1335: loss 0.043412
batch 1336: loss 0.026571
batch 1337: loss 0.003303
batch 1338: loss 0.009227
batch 1339: loss 0.013370
batch 1340: loss 0.090468
batch 1341: loss 0.106231
batch 1342: loss 0.020977
batch 1343: loss 0.082000
batch 1344: 

...

batch 4470: loss 0.001137
batch 4471: loss 0.000242
batch 4472: loss 0.007737
batch 4473: loss 0.004405
batch 4474: loss 0.003351
batch 4475: loss 0.015637
batch 4476: loss 0.008982
batch 4477: loss 0.008071
batch 4478: loss 0.004253
batch 4479: loss 0.000214
batch 4480: loss 0.002624
batch 4481: loss 0.000256
batch 4482: loss 0.016110
batch 4483: loss 0.007845
batch 4484: loss 0.006601
batch 4485: loss 0.017587
batch 4486: loss 0.000259
batch 4487: loss 0.000247
batch 4488: loss 0.001047
batch 4489: loss 0.000050
batch 4490: loss 0.000036
batch 4491: loss 0.000421
batch 4492: loss 0.001345
batch 4493: loss 0.000131
batch 4494: loss 0.001780
batch 4495: loss 0.009587
batch 4496: loss 0.000458
batch 4497: loss 0.008176
batch 4498: loss 0.000045
batch 4499: loss 0.000032
batch 4500: loss 0.002105
batch 4501: loss 0.000196
batch 4502: loss 0.000391
batch 4503: loss 0.286837
batch 4504: loss 0.000092
batch 4505: loss 0.000025
batch 4506: loss 0.009855
batch 4507: loss 0.001316
batch 4508: 

batch 4786: loss 0.000309
batch 4787: loss 0.003894
batch 4788: loss 0.004536
batch 4789: loss 0.096692
batch 4790: loss 0.083430
batch 4791: loss 0.002429
batch 4792: loss 0.000066
batch 4793: loss 0.002277
batch 4794: loss 0.003766
batch 4795: loss 0.003593
batch 4796: loss 0.010558
batch 4797: loss 0.001105
batch 4798: loss 0.000134
batch 4799: loss 0.005035
batch 4800: loss 0.010982
batch 4801: loss 0.000438
batch 4802: loss 0.001177
batch 4803: loss 0.000351
batch 4804: loss 0.085081
batch 4805: loss 0.001075
batch 4806: loss 0.081217
batch 4807: loss 0.000279
batch 4808: loss 0.002927
batch 4809: loss 0.003391
batch 4810: loss 0.024663
batch 4811: loss 0.003395
batch 4812: loss 0.001622
batch 4813: loss 0.001495
batch 4814: loss 0.001568
batch 4815: loss 0.000403
batch 4816: loss 0.004037
batch 4817: loss 0.010839
batch 4818: loss 0.002098
batch 4819: loss 0.002163
batch 4820: loss 0.000915
batch 4821: loss 0.052040
batch 4822: loss 0.015248
batch 4823: loss 0.000316
batch 4824: 

batch 5102: loss 0.003953
batch 5103: loss 0.045331
batch 5104: loss 0.017277
batch 5105: loss 0.023401
batch 5106: loss 0.028257
batch 5107: loss 0.012053
batch 5108: loss 0.012236
batch 5109: loss 0.001361
batch 5110: loss 0.001931
batch 5111: loss 0.014954
batch 5112: loss 0.110760
batch 5113: loss 0.015987
batch 5114: loss 0.000367
batch 5115: loss 0.056315
batch 5116: loss 0.001057
batch 5117: loss 0.029216
batch 5118: loss 0.004467
batch 5119: loss 0.008103
batch 5120: loss 0.026591
batch 5121: loss 0.011143
batch 5122: loss 0.005864
batch 5123: loss 0.012485
batch 5124: loss 0.011937
batch 5125: loss 0.079579
batch 5126: loss 0.035045
batch 5127: loss 0.001514
batch 5128: loss 0.004110
batch 5129: loss 0.000169
batch 5130: loss 0.002765
batch 5131: loss 0.010516
batch 5132: loss 0.007763
batch 5133: loss 0.001216
batch 5134: loss 0.001742
batch 5135: loss 0.000963
batch 5136: loss 0.000717
batch 5137: loss 0.002058
batch 5138: loss 0.045151
batch 5139: loss 0.009852
batch 5140: 

batch 5418: loss 0.000263
batch 5419: loss 0.001464
batch 5420: loss 0.002761
batch 5421: loss 0.000616
batch 5422: loss 0.003167
batch 5423: loss 0.001170
batch 5424: loss 0.025071
batch 5425: loss 0.000977
batch 5426: loss 0.161129
batch 5427: loss 0.014186
batch 5428: loss 0.001157
batch 5429: loss 0.003631
batch 5430: loss 0.002277
batch 5431: loss 0.066537
batch 5432: loss 0.000309
batch 5433: loss 0.000532
batch 5434: loss 0.001494
batch 5435: loss 0.000581
batch 5436: loss 0.032606
batch 5437: loss 0.003435
batch 5438: loss 0.005100
batch 5439: loss 0.000069
batch 5440: loss 0.002091
batch 5441: loss 0.001390
batch 5442: loss 0.007469
batch 5443: loss 0.001580
batch 5444: loss 0.000759
batch 5445: loss 0.001353
batch 5446: loss 0.001528
batch 5447: loss 0.042433
batch 5448: loss 0.001374
batch 5449: loss 0.001195
batch 5450: loss 0.032046
batch 5451: loss 0.003610
batch 5452: loss 0.009860
batch 5453: loss 0.005777
batch 5454: loss 0.000295
batch 5455: loss 0.028818
batch 5456: 

batch 5734: loss 0.054153
batch 5735: loss 0.001337
batch 5736: loss 0.000111
batch 5737: loss 0.000023
batch 5738: loss 0.000114
batch 5739: loss 0.001501
batch 5740: loss 0.000071
batch 5741: loss 0.000044
batch 5742: loss 0.000833
batch 5743: loss 0.201666
batch 5744: loss 0.122161
batch 5745: loss 0.002013
batch 5746: loss 0.006090
batch 5747: loss 0.000150
batch 5748: loss 0.009275
batch 5749: loss 0.000055
batch 5750: loss 0.001751
batch 5751: loss 0.000221
batch 5752: loss 0.005397
batch 5753: loss 0.032653
batch 5754: loss 0.000069
batch 5755: loss 0.000180
batch 5756: loss 0.000010
batch 5757: loss 0.004644
batch 5758: loss 0.004998
batch 5759: loss 0.010354
batch 5760: loss 0.000098
batch 5761: loss 0.000267
batch 5762: loss 0.000064
batch 5763: loss 0.000019
batch 5764: loss 0.000049
batch 5765: loss 0.000020
batch 5766: loss 0.021671
batch 5767: loss 0.023187
batch 5768: loss 0.007757
batch 5769: loss 0.006233
batch 5770: loss 0.000046
batch 5771: loss 0.000413
batch 5772: 

In [6]:
sparse_categorical_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()
num_batches = int(data_loader.num_test_data // batch_size)
for batch_index in range(num_batches):
    start_index, end_index = batch_index * batch_size, (batch_index + 1) * batch_size
    y_pred = model.predict(data_loader.test_data[start_index: end_index])
    sparse_categorical_accuracy.update_state(y_true=data_loader.test_label[start_index: end_index], y_pred=y_pred)
print("test accuracy: %f" % sparse_categorical_accuracy.result())

test accuracy: 0.990600
